# Часть 1. Pandas

In [1]:
import pandas as pd

In [2]:
ratings_col = ['user_id', 'item_id', 'rating', 'timestamp']
movies_col = ['movie_id', 'movie_title', 'release_date', 'video_release_date', 'imdb_url']
genres_col = ['unknown', 'action', 'adventure', 'animation', 'children', 
              'comedy', 'crime', 'documentary', 'drama', 'fantasy', 
              'film-noir', 'horror', 'musical', 'mystery', 'romance',
              'sci-fi', 'thriller', 'war', 'western']

In [3]:
df_ratings = pd.read_csv('u.data.csv', sep='\t', names=ratings_col)
df_movies = pd.read_csv('u.item.csv', sep='|', names=movies_col+genres_col, encoding='latin-1')

In [4]:
max_rating_user = df_ratings.groupby('user_id')['rating'].count().sort_values(ascending=False).reset_index().iloc[0, 0]
max_rating_user

405

In [5]:
df_new_ratings = df_ratings[df_ratings.user_id == max_rating_user]
df_new_ratings

,user_id,item_id,rating,timestamp
12276,405,56,4,885544911
12383,405,592,1,885548670
12430,405,1582,1,885548670
12449,405,171,1,885549544
12460,405,580,1,885547447
...,...,...,...,...
98956,405,375,1,885546835
98978,405,445,4,885548435
99148,405,1246,1,885547735
99465,405,196,1,885546112


In [6]:
df_movies['year'] = pd.to_numeric(
    df_movies.release_date.str.slice(start=-4),
    errors='coerce'
)

In [7]:
# Есть одно пустое значение - заполним его
df_movies.fillna(
    value=df_movies['year'].median(),
    inplace=True
)

In [8]:
df_new_ratings = df_new_ratings.merge(
    df_ratings.groupby('item_id').agg(
        rating_count = ('rating', 'count'), 
        rating_mean = ('rating', 'mean')
    ),
    how='left',
    left_on='item_id',
    right_index=True
).merge(
    df_movies[['movie_id', 'year']+genres_col],
    how='left',
    left_on='item_id',
    right_on='movie_id'

)
df_new_ratings.head()

,user_id,item_id,rating,timestamp,rating_count,rating_mean,movie_id,year,unknown,action,...,fantasy,film-noir,horror,musical,mystery,romance,sci-fi,thriller,war,western
0,405,56,4,885544911,394,4.060914,56,1994.0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,405,592,1,885548670,9,3.333333,592,1995.0,0,0,...,0,0,0,0,1,0,0,1,0,0
2,405,1582,1,885548670,1,1.000000,1582,1947.0,0,0,...,0,1,0,0,0,0,0,0,0,0
3,405,171,1,885549544,65,3.876923,171,1991.0,0,0,...,0,0,0,0,0,0,1,0,0,0
4,405,580,1,885547447,32,3.375000,580,1995.0,0,0,...,0,0,0,0,0,1,0,0,0,0


In [9]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score


In [10]:
X_test, X_train, y_test, y_train = train_test_split(
    df_new_ratings[['year', 'rating_count', 'rating_mean']+ genres_col],
    df_new_ratings['rating'],
    random_state=42
)

In [11]:
lr = LinearRegression().fit(X_train, y_train)


In [12]:
lr.score(X_train, y_train)

0.2573310972549465

In [13]:
lr.score(X_test, y_test)

0.29369532380547103

Так себе модель получилась

# Часть 2. Spark

In [14]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.master("local[*]").appName("itog").getOrCreate()

24/04/26 17:29:54 WARN Utils: Your hostname, MacBook-Pro-Vadim.local resolves to a loopback address: 127.0.0.1; using 192.168.139.65 instead (on interface en0)
24/04/26 17:29:54 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/04/26 17:30:09 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [27]:
dfs_ratings = spark.read.option('delimiter', '\t').csv(
    'u.data.csv', 
    header=False, 
    inferSchema=True
    ).toDF(*ratings_col)

dfs_movies = spark.read.option('delimiter', '|').csv(
    'u.item.csv', 
    header=False, 
    inferSchema=True).toDF(*(movies_col + genres_col))

In [28]:
dfs_ratings.show(5)

+-------+-------+------+---------+
|user_id|item_id|rating|timestamp|
+-------+-------+------+---------+
|    196|    242|     3|881250949|
|    186|    302|     3|891717742|
|     22|    377|     1|878887116|
|    244|     51|     2|880606923|
|    166|    346|     1|886397596|
+-------+-------+------+---------+
only showing top 5 rows



In [29]:
dfs_movies.show(5)

+--------+-----------------+------------+------------------+--------------------+-------+------+---------+---------+--------+------+-----+-----------+-----+-------+---------+------+-------+-------+-------+------+--------+---+-------+
|movie_id|      movie_title|release_date|video_release_date|            imdb_url|unknown|action|adventure|animation|children|comedy|crime|documentary|drama|fantasy|film-noir|horror|musical|mystery|romance|sci-fi|thriller|war|western|
+--------+-----------------+------------+------------------+--------------------+-------+------+---------+---------+--------+------+-----+-----------+-----+-------+---------+------+-------+-------+-------+------+--------+---+-------+
|       1| Toy Story (1995)| 01-Jan-1995|              NULL|http://us.imdb.co...|      0|     0|        0|        1|       1|     1|    0|          0|    0|      0|        0|     0|      0|      0|      0|     0|       0|  0|      0|
|       2| GoldenEye (1995)| 01-Jan-1995|              NULL|http

In [35]:
dfs_ratings_movies = dfs_ratings.join(
    dfs_movies,
    dfs_ratings.item_id == dfs_movies.movie_id
)
dfs_ratings_movies.show(5)

24/04/26 18:19:37 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


+-------+-------+------+---------+--------+--------------------+------------+------------------+--------------------+-------+------+---------+---------+--------+------+-----+-----------+-----+-------+---------+------+-------+-------+-------+------+--------+---+-------+
|user_id|item_id|rating|timestamp|movie_id|         movie_title|release_date|video_release_date|            imdb_url|unknown|action|adventure|animation|children|comedy|crime|documentary|drama|fantasy|film-noir|horror|musical|mystery|romance|sci-fi|thriller|war|western|
+-------+-------+------+---------+--------+--------------------+------------+------------------+--------------------+-------+------+---------+---------+--------+------+-----+-----------+-----+-------+---------+------+-------+-------+-------+------+--------+---+-------+
|    196|    242|     3|881250949|     242|        Kolya (1996)| 24-Jan-1997|              NULL|http://us.imdb.co...|      0|     0|        0|        0|       0|     1|    0|          0|    

In [39]:
# Можно группировать по item_id. Зависит от того, что нужно id или название
dfs_ratings_movies.groupBy('movie_title').agg({'rating':'mean'}).show()

+--------------------+------------------+
|         movie_title|       avg(rating)|
+--------------------+------------------+
|   Annie Hall (1977)| 3.911111111111111|
|Heavenly Creature...|3.6714285714285713|
|       Psycho (1960)| 4.100418410041841|
|Snow White and th...|3.7093023255813953|
|Night of the Livi...|          3.421875|
|When We Were King...| 4.045454545454546|
| If Lucy Fell (1996)|2.7586206896551726|
|    Fair Game (1995)|2.1818181818181817|
| Three Wishes (1995)|3.2222222222222223|
|         Cosi (1996)|               4.0|
|Paris, France (1993)|2.3333333333333335|
|Spanking the Monk...| 3.074074074074074|
|I'll Do Anything ...|               2.6|
|        Mondo (1996)|               3.0|
| Evil Dead II (1987)|3.5168539325842696|
|    Threesome (1994)| 2.838709677419355|
|Last Action Hero ...|2.7457627118644066|
|Reality Bites (1994)| 2.961038961038961|
|Colonel Chabert, ...|               3.5|
|   Blue Chips (1994)|2.6666666666666665|
+--------------------+------------

In [54]:
dfs_ratings_movies.unpivot(
    'rating', genres_col, 'ganre', 'flag'
).filter('flag !=0').groupBy('ganre').avg('rating').show()

+-----------+------------------+
|      ganre|       avg(rating)|
+-----------+------------------+
|      crime|3.6322780881440098|
|    fantasy|3.2152366863905324|
|documentary|3.6728232189973613|
|    unknown|               3.2|
|     action| 3.480245417953027|
|  animation|3.5766990291262135|
|    mystery|  3.63813155386082|
|     horror|3.2903893172841827|
|  film-noir|3.9215233698788228|
|    musical| 3.521396851029471|
|  adventure| 3.503526503308369|
|      drama|3.6873793708484772|
|    western|3.6132686084142396|
|   children|3.3532442216652742|
|        war| 3.815811874866993|
|    romance| 3.621704948358255|
|   thriller|3.5090069495245064|
|     sci-fi|3.5607227022780834|
|     comedy|3.3940734781442745|
+-----------+------------------+



In [57]:
dfs_ratings_movies.groupBy('movie_title').count().sort('count', ascending=False).limit(5).show()

+--------------------+-----+
|         movie_title|count|
+--------------------+-----+
|    Star Wars (1977)|  583|
|      Contact (1997)|  509|
|        Fargo (1996)|  508|
|Return of the Jed...|  507|
|    Liar Liar (1997)|  485|
+--------------------+-----+



In [58]:
dfs_ratings_movies.groupBy('movie_title').count().sort('count').limit(5).show()

+--------------------+-----+
|         movie_title|count|
+--------------------+-----+
|Vie est belle, La...|    1|
|Next Step, The (1...|    1|
|JLG/JLG - autopor...|    1|
|Modern Affair, A ...|    1|
|       Target (1995)|    1|
+--------------------+-----+

